# 실제 데이터 풀로 가져오기

In [45]:
import pandas as pd
import yfinance as yf
start = '2021-01-01'
end = '2024-12-31'
symbol = 'AAPL' # "005930.KS"
df = yf.download(symbol, start, end)
df = df.rename(columns={'Adj Close': 'Adj_Close'})
# df['ds'] = pd.to_datetime(df.index, format = '%Y-%m-%d') # 굳이 넣어야하나 고민중
# df['y'] = df['Close'] # 마찬가지로 굳이 넣어야하나 고민중
df.head(3)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2021-01-04,133.520004,133.610001,126.760002,129.410004,127.002083,143301900
2021-01-05,128.889999,131.740005,128.429993,131.009995,128.572327,97664900
2021-01-06,127.720001,131.050003,126.379997,126.599998,124.244362,155088000


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 800 entries, 2021-01-04 to 2024-03-08
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       800 non-null    float64
 1   High       800 non-null    float64
 2   Low        800 non-null    float64
 3   Close      800 non-null    float64
 4   Adj_Close  800 non-null    float64
 5   Volume     800 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 43.8 KB


In [47]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'stockdata' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'aapl_from2021-01-01' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
df.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<00:00, 999.12it/s]


# 주요 지수 적재

## S&P

In [1]:
import pandas as pd
import yfinance as yf
spy_ticker = "SPY"

# start와 end date 사이의 정보를 가져옵니다.
spy = yf.download(spy_ticker, start="2021-01-01", end="2023-12-31")
spy = spy.rename(columns={'Adj Close': 'Adj_Close'})
spy

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2021-01-04,375.309998,375.450012,364.820007,368.790009,352.767212,110210800
2021-01-05,368.100006,372.500000,368.049988,371.329987,355.196747,66426200
2021-01-06,369.709991,376.980011,369.119995,373.549988,357.320312,107997700
2021-01-07,376.100006,379.899994,375.910004,379.100006,362.629211,68766800
2021-01-08,380.589996,381.489990,377.100006,381.260010,364.695374,71677200
...,...,...,...,...,...,...
2023-12-22,473.859985,475.380005,471.700012,473.649994,473.649994,67126600
2023-12-26,474.070007,476.579987,473.989990,475.649994,475.649994,55387000
2023-12-27,475.440002,476.660004,474.890015,476.510010,476.510010,68000300


In [2]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'major_indices' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'spy' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
spy.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


## nasdoc

In [3]:
import pandas as pd
import yfinance as yf
# 나스닥 종합 지수로 나스닥 정보를 가져옵니다
nasdaq_ticker = "^IXIC"

# start와 end date 사이의 정보를 가져옵니다.
nasdaq_data = yf.download(nasdaq_ticker, start="2021-01-01", end="2023-12-31")
nasdaq_data = nasdaq_data.rename(columns={'Adj Close': 'Adj_Close'})
nasdaq_data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2021-01-04,12958.519531,12958.719727,12543.240234,12698.450195,12698.450195,6636170000
2021-01-05,12665.650391,12828.269531,12665.650391,12818.959961,12818.959961,6971860000
2021-01-06,12666.150391,12909.629883,12649.990234,12740.790039,12740.790039,7689880000
2021-01-07,12867.339844,13090.910156,12867.339844,13067.480469,13067.480469,6841480000
2021-01-08,13160.219727,13208.089844,13036.549805,13201.980469,13201.980469,7289390000
...,...,...,...,...,...,...
2023-12-22,15006.179688,15047.190430,14927.120117,14992.969727,14992.969727,4796600000
2023-12-26,15028.690430,15101.179688,15024.059570,15074.570312,15074.570312,6120600000
2023-12-27,15089.660156,15114.080078,15051.669922,15099.179688,15099.179688,7480170000


In [4]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'major_indices' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'nasdaq' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
nasdaq_data.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


# prophet 예측값 arima 예측값 union시키기

## 아리마 예측 데이터 불러오기

In [1]:
# 아리마 예측 적재용
import pandas as pd
df_arima = pd.read_csv('arima_forecast.csv')# 'ARIMA_forecast_2024-01.csv')

df_arima['ds'] = pd.to_datetime(df_arima['ds'])

# df_arima = df_arima.rename(columns={'Predicted Price': 'Predicted_Price'})
# df_arima = df_arima.rename(columns={'Upper Bound': 'Upper_Bound'})
# df_arima = df_arima.rename(columns={'Lower Bound': 'Lower_Bound'})

In [13]:
df_arima.head(3)

,ds,yhat,yhat_lower,yhat_upper
0,2021-01-04,188.039993,181.956899,194.123088
1,2021-01-05,129.410004,122.663076,136.156931
2,2021-01-06,131.009995,124.264746,137.755243


In [14]:

from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'stockdata' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'predict_aapl_arima' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
df_arima.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


## prophet 예측데이터 만들기

In [2]:
import pandas as pd
import yfinance as yf
start = '2021-01-01'
end = '2023-12-31'
symbol = 'AAPL' # "005930.KS"
df_merge = yf.download(symbol, start, end)
df_merge = df_merge.rename(columns={'Adj Close': 'Adj_Close'})
# df_merge['ds'] = pd.to_datetime(df_merge.index, format = '%Y-%m-%d') # 굳이 넣어야하나 고민중
# df_merge['y'] = df_merge['Close'] # 마찬가지로 굳이 넣어야하나 고민중
df_merge.head(3)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2021-01-04,133.520004,133.610001,126.760002,129.410004,127.002083,143301900
2021-01-05,128.889999,131.740005,128.429993,131.009995,128.572342,97664900
2021-01-06,127.720001,131.050003,126.379997,126.599998,124.244385,155088000


In [3]:
import numpy as np
from prophet import Prophet
df_merge['ds'] = pd.to_datetime(df_merge.index, format = '%Y-%m-%d') # 굳이 넣어야하나 고민중
df_merge['y'] = df_merge['Close'] # 마찬가지로 굳이 넣어야하나 고민중
df_input = df_merge[['ds','y']]
model_prophet = Prophet(changepoint_prior_scale = 0.15, daily_seasonality = True)
model_prophet.fit(df_input)
fcast_time = 365 # 365일 예측
freq_option = 'D'
future = model_prophet.make_future_dataframe(periods = fcast_time, freq = freq_option)
from workalendar.usa import UnitedStates
# AAPL의 경우 미국 거래소에서 거래가 이루어지니 미국 공휴일 정보 가져와서 리스트로 만드는 과정
# cal = workalendar.usa.UnitedStates()
cal = UnitedStates()
holiday_dates = []

for year in range(2021, 2025):  # 2021년부터 2024년까지
    holiday_dates.extend([date for date, _ in cal.holidays(year)])
    
future = future[future['ds'].dt.dayofweek < 5] # 주말 제외 작업
future = future[~future['ds'].isin(holiday_dates)] # 공휴일 제외 작업
df_forecast = model_prophet.predict(future)

15:15:08 - cmdstanpy - INFO - Chain [1] start processing
15:15:09 - cmdstanpy - INFO - Chain [1] done processing


In [17]:
df_forecast.head(3)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2021-01-04,125.837293,127.921738,138.832164,125.837293,125.837293,7.130274,7.130274,7.130274,2.360639,...,0.160907,0.160907,0.160907,4.608728,4.608728,4.608728,0.0,0.0,0.0,132.967567
1,2021-01-05,125.788028,127.104649,138.156965,125.788028,125.788028,6.900233,6.900233,6.900233,2.360639,...,0.143280,0.143280,0.143280,4.396314,4.396314,4.396314,0.0,0.0,0.0,132.688260
2,2021-01-06,125.738762,126.867076,137.820493,125.738762,125.738762,6.693765,6.693765,6.693765,2.360639,...,0.150670,0.150670,0.150670,4.182456,4.182456,4.182456,0.0,0.0,0.0,132.432527


In [18]:
df_merge.head(3)

,Open,High,Low,Close,Adj_Close,Volume,ds,y
Date,,,,,,,,
2021-01-04,133.520004,133.610001,126.760002,129.410004,127.002090,143301900,2021-01-04,129.410004
2021-01-05,128.889999,131.740005,128.429993,131.009995,128.572327,97664900,2021-01-05,131.009995
2021-01-06,127.720001,131.050003,126.379997,126.599998,124.244377,155088000,2021-01-06,126.599998


In [9]:
df_arima.head(3)

,ds,yhat,yhat_upper,yhat_lower
0,2024-01-01,192.529999,196.804993,188.255005
1,2024-01-02,192.529999,196.803082,188.256916
2,2024-01-03,192.529999,196.801174,188.258824


In [4]:
df_merge.reset_index(drop=True, inplace=True)

In [5]:
df_close = df_merge[['ds','Close']]
df_close.head(3)

,ds,Close
0,2021-01-04,129.410004
1,2021-01-05,131.009995
2,2021-01-06,126.599998


In [6]:
df_prophet_concat=df_forecast[['ds','yhat','yhat_lower','yhat_upper']]
df_prophet_concat.head(3)

,ds,yhat,yhat_lower,yhat_upper
0,2021-01-04,132.967567,127.109232,138.286348
1,2021-01-05,132.688260,127.433978,138.283137
2,2021-01-06,132.432527,126.758090,138.099001


In [7]:
df_prophet_concat2 = pd.merge(df_prophet_concat, df_close, on='ds', how='left')
df_prophet_concat2.iloc[-252:-248]
df_prophet_concat2.info()
df_prophet_concat2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 996 entries, 0 to 995
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ds          996 non-null    datetime64[ns]
 1   yhat        996 non-null    float64       
 2   yhat_lower  996 non-null    float64       
 3   yhat_upper  996 non-null    float64       
 4   Close       746 non-null    float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 39.0 KB


,ds,yhat,yhat_lower,yhat_upper,Close
0,2021-01-04,132.967567,127.109232,138.286348,129.410004
1,2021-01-05,132.688260,127.433978,138.283137,131.009995
2,2021-01-06,132.432527,126.758090,138.099001,126.599998
3,2021-01-07,132.025325,126.653870,137.419581,130.919998
4,2021-01-08,131.901294,126.508503,137.346006,132.050003
...,...,...,...,...,...
991,2024-12-20,208.399744,37.631154,360.887273,NaN
992,2024-12-23,208.497468,34.138863,363.249018,NaN
993,2024-12-24,208.480149,33.919080,363.928373,NaN
994,2024-12-26,208.297338,34.393119,363.577510,NaN


In [8]:
df_arima_concat2 = pd.merge(df_arima, df_close, on='ds', how='left') # df_arima_concat 은 없는게 맞음 애초에 컬럼이 그것들 뿐이라 안만듦
df_arima_concat2.iloc[-22:-18]
df_arima_concat2

,ds,yhat,yhat_lower,yhat_upper,Close
0,2021-01-04,188.039993,181.956899,194.123088,129.410004
1,2021-01-05,129.410004,122.663076,136.156931,131.009995
2,2021-01-06,131.009995,124.264746,137.755243,126.599998
3,2021-01-07,126.599998,119.853352,133.346645,130.919998
4,2021-01-08,130.919998,124.172099,137.667897,132.050003
...,...,...,...,...,...
768,2024-01-24,195.179993,188.896540,201.463446,NaN
769,2024-01-25,194.500000,188.217844,200.782156,NaN
770,2024-01-26,194.169998,187.889184,200.450812,NaN
771,2024-01-29,192.419998,186.140136,198.699860,NaN


In [34]:
df_arima_concat2.sort_values(by='ds',inplace=True)

In [38]:
# df_arima_concat2.reset_index(drop=True, inplace=True)

In [9]:
df_prophet_concat2['model']='prophet'

In [10]:
df_arima_concat2['model']='arima'

In [13]:
combined_data = pd.concat([df_prophet_concat2, df_arima_concat2])
combined_data.reset_index(drop=True, inplace=True)

In [20]:
combined_data.iloc[-40:]

,ds,yhat,yhat_lower,yhat_upper,Close,model
1729,2023-12-01,189.949997,183.644118,196.255876,191.240005,arima
1730,2023-12-04,191.240005,184.935288,197.544723,189.429993,arima
1731,2023-12-05,189.429993,183.126220,195.733766,193.419998,arima
1732,2023-12-06,193.419998,187.115387,199.724609,192.320007,arima
1733,2023-12-07,192.320007,186.016709,198.623305,194.270004,arima
1734,2023-12-08,194.270004,187.967580,200.572429,195.710007,arima
1735,2023-12-11,195.710007,189.408685,202.011328,193.179993,arima
1736,2023-12-12,193.179993,186.879404,199.480581,194.710007,arima
1737,2023-12-13,194.710007,188.410276,201.009737,197.960007,arima
1738,2023-12-14,197.960007,191.660276,204.259737,198.110001,arima


In [23]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'stockdata' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'combined_data' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
combined_data.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


In [22]:
combined_data.to_csv('combined_data2.csv')

# prophet & arima 예측 평가 union

## 아리마 예측 평가 적재용

In [ ]:
# 아리마 예측 적재용
import pandas as pd
metrics_arima = pd.read_csv('metrics_arima.csv')

In [ ]:

from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'predict_metrics' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'metrics_aapl_arima' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
metrics_arima.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

## prophet 위 arima랑 같은 방식으로 해서 적재하는 부분까지

## 아리마 불러오기 및 적재

In [10]:
# 아리마 예측 적재용
import pandas as pd
metrics_arima = pd.read_csv('metrics_arima.csv')

In [ ]:

from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'predict_metrics' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'metrics_aapl_arima' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
metrics_arima.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<00:00, 999.12it/s]


## prophet 불러오기 및 예측 및 적재

In [25]:
import pandas as pd
import yfinance as yf
start = '2022-01-01'
end = '2023-12-31'
symbol = 'AAPL' # "005930.KS"
total_train_df = yf.download(symbol, start, end)
total_train_df = total_train_df.rename(columns={'Adj Close': 'Adj_Close'})
total_train_df['ds'] = pd.to_datetime(total_train_df.index, format = '%Y-%m-%d')
total_train_df['y'] = total_train_df['Close']

[*********************100%%**********************]  1 of 1 completed


In [ ]:
# fcast_time = 11 #365 # 365일 예측
# freq_option = 'D'
# us_holidays = make_holidays_df(
#     year_list=[2022 + i for i in range(3)], country='US'
# )

In [26]:
total_train_df_input = total_train_df[['ds','y']]

In [27]:
train_size = int(len(total_train_df_input) * 0.9)
df_train = total_train_df_input[:train_size]
df_test = total_train_df_input[train_size:]

In [28]:
model_prophet = Prophet(changepoint_prior_scale=0.15, daily_seasonality = True) # , holidays=us_holidays)
model_prophet.fit(df_train)
forecast_test = model_prophet.predict(df_test)

17:46:33 - cmdstanpy - INFO - Chain [1] start processing
17:46:33 - cmdstanpy - INFO - Chain [1] done processing


In [29]:
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

mse = mean_squared_error(df_test['y'], forecast_test['yhat'])
print('MSE: ', mse)
mae = mean_absolute_error(df_test['y'], forecast_test['yhat'])
print('MAE: ', mae)
rmse = math.sqrt(mean_squared_error(df_test['y'], forecast_test['yhat']))
print('RMSE: ', rmse)
mape = mean_absolute_percentage_error(df_test['y'], forecast_test['yhat'])
print('MAPE: ' , '{:.2f}%'.format(mape*100))
r2score = r2_score(df_test['y'], forecast_test['yhat'])
print('r2 score: ', r2score)

MSE:  241.61045606234183
MAE:  13.674012598803841
RMSE:  15.543823727202449
MAPE:  7.18%
r2 score:  -1.973770022074588


In [30]:
metrics = pd.DataFrame(
    {
    'MSE': [mse],
    'MAE': [mae],
    'RMSE': [rmse],
    'MAPE_percentage': [mape*100]
    }
)

In [31]:
metrics.to_csv('metrics_prophet_prototype.csv',index=False)

## union

In [ ]:
# metrics_prophet = pd.read_csv('metrics_prophet1.csv')
metrics_prophet = metrics

# prophet 적당한 파라미터로으로 다시 예측하여 예측값과 평가지표 적재

# 지호님 기사 긁어 온 부분 적재

## 기사 긁어오는 코드

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def job(page_num):
    url = f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EC%95%A0%ED%94%8C&start={page_num}"
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    news_items = soup.select('.news_area')

    news_info = [{'title': item.select_one('.news_tit').get_text(), 'url': item.select_one('.news_tit').get('href')} for item in news_items]
    return news_info

news_info = []
for i in range(1, 101, 10):
    news_info.extend(job(i))

df_article = pd.DataFrame(news_info)

In [5]:
df_article.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   90 non-null     object
 1   url     90 non-null     object
dtypes: object(2)
memory usage: 1.5+ KB


In [9]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'major_indices' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'naver_article' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
df_article.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


# 최종 prophet 예측 데이터 결정

## import

In [2]:
import math
from datetime import datetime, timedelta

import pandas as pd
import yfinance as yf
import numpy as np
from tqdm.auto import tqdm
from prophet import Prophet
from prophet.make_holidays import make_holidays_df
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

## 예측 평가용 (21~23년 학습 한달 예측)

### 예측 및 그래프

In [6]:
start = '2021-01-01' # '2022-01-01'
end = '2023-12-31'
symbol = 'AAPL' # "005930.KS"
prophet_train_df = yf.download(symbol, start, end)
prophet_train_df = prophet_train_df.rename(columns={'Adj Close': 'Adj_Close'})
prophet_train_df['ds'] = pd.to_datetime(prophet_train_df.index, format = '%Y-%m-%d')
prophet_train_df['y'] = prophet_train_df['Close']

prophet_train_df_input = prophet_train_df[['ds','y']]

fcast_time = 30 #365 # 365일 예측
freq_option = 'D'


# 문자열을 datetime 객체로 변환
end_date = datetime.strptime(end, '%Y-%m-%d')
start_date_r = end_date + timedelta(days=1) # fcast_time+1 만큼의 일수를 더함
end_date_r = end_date + timedelta(days=fcast_time+2) # fcast_time+1 만큼의 일수를 더함 end날 이전까지 가져와서 +1 해줌
start_r = start_date_r.strftime('%Y-%m-%d') # 문자로 바꿔줌
end_r = end_date_r.strftime('%Y-%m-%d') # 문자로 바꿔줌
ticker_r = 'AAPL'

df_recent = yf.download(ticker_r, start=start_r, end=end_r)
df_recent['ds'] = pd.to_datetime(df_recent.index, format = '%Y-%m-%d')
df_recent['y'] = df_recent['Close']

us_holidays = make_holidays_df(
    year_list=[2021 + i for i in range(4)], country='US'
)

model_prophet = Prophet(daily_seasonality = True, holidays=us_holidays, changepoint_prior_scale=0.3,
                        holidays_prior_scale=1.0, n_changepoints=50, seasonality_prior_scale=0.01).fit(prophet_train_df_input)

future = model_prophet.make_future_dataframe(periods = fcast_time+3, freq = freq_option)
future = future[(future['ds'].dt.dayofweek < 5)&(~future['ds'].isin(us_holidays['ds']))] # 주말 제외 작업 & # 공휴일 제외 작업

df_forecast = model_prophet.predict(future)
# model_prophet.plot(df_forecast, xlabel = 'Date',ylabel ='price($)')
# model_prophet.plot_components(df_forecast);

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


14:02:08 - cmdstanpy - INFO - Chain [1] start processing
14:02:09 - cmdstanpy - INFO - Chain [1] done processing


In [25]:
df_forecast.to_csv('prophet한달치_예상값.csv',index=False)

### 예측 평가

In [7]:
df_forecast.set_index('ds', inplace=True, drop=False)
fc = df_forecast[df_forecast['ds'].isin(df_recent['ds'])]['yhat']

mse = mean_squared_error(df_recent['y'], fc)
mae = mean_absolute_error(df_recent['y'], fc)
rmse = math.sqrt(mean_squared_error(df_recent['y'], fc))
mape = mean_absolute_percentage_error(df_recent['y'], fc)
r2score = r2_score(df_recent['y'],fc)
print(f'mse: {mse}, mae: {mae}, rmse: {rmse}, mape: {mape}, r2_score: {r2score}')

mse: 38.85967063137697, mae: 5.202402911757141, rmse: 6.233752532093087, mape: 0.028079003006489328, r2_score: -1.0268363215110603


In [8]:
metrics = pd.DataFrame(
    {
    'MSE': [mse],
    'MAE': [mae],
    'RMSE': [rmse],
    'MAPE_percentage': [mape*100]
    }
)

### 적재

In [9]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'predict_metrics' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'metrics_prophet_aapl' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
metrics.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


In [11]:
metrics.to_csv('metrics_prophet(final).csv',index=False)

## arima랑 union시키기

In [15]:
# metrics_prophet = metrics
metrics_prophet =pd.read_csv('metrics_prophet(final).csv')

In [ ]:
# metrics_prophet = metrics
metrics_prophet =pd.read_csv('metrics_prophet(final).csv')

In [ ]:
metrics_prophet = metrics_prophet.assign(model='prophet')
metrics_arima = metrics_arima.assign(model='arima')

In [ ]:
metrics_combined = pd.concat([metrics_prophet,metrics_arima])

In [ ]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'predict_metrics' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'metrics_combined_aapl' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
metrics_combined.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


In [17]:
metrics_prophet = metrics_prophet.assign(model='prophet')
metrics_arima = metrics_arima.assign(model='arima')

In [21]:
metrics_combined = pd.concat([metrics_prophet,metrics_arima])

In [23]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'predict_metrics' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'metrics_combined_aapl' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
metrics_combined.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


## 예측 길게

In [21]:
start = '2021-01-01' # '2022-01-01'
end = '2023-12-31'
symbol = 'AAPL' # "005930.KS"
prophet_train_df = yf.download(symbol, start, end)
prophet_train_df = prophet_train_df.rename(columns={'Adj Close': 'Adj_Close'})
prophet_train_df['ds'] = pd.to_datetime(prophet_train_df.index, format = '%Y-%m-%d')
prophet_train_df['y'] = prophet_train_df['Close']

prophet_train_df_input = prophet_train_df[['ds','y']]

fcast_time = 365 #365 # 365일 예측
freq_option = 'D'


# 문자열을 datetime 객체로 변환
end_date = datetime.strptime(end, '%Y-%m-%d')
start_date_r = end_date + timedelta(days=1) # fcast_time+1 만큼의 일수를 더함
end_date_r = end_date + timedelta(days=fcast_time+2) # fcast_time+1 만큼의 일수를 더함 end날 이전까지 가져와서 +1 해줌
start_r = start_date_r.strftime('%Y-%m-%d') # 문자로 바꿔줌
end_r = end_date_r.strftime('%Y-%m-%d') # 문자로 바꿔줌
ticker_r = 'AAPL'

df_recent = yf.download(ticker_r, start=start_r, end=end_r)
df_recent['ds'] = pd.to_datetime(df_recent.index, format = '%Y-%m-%d')
df_recent['y'] = df_recent['Close']

us_holidays = make_holidays_df(
    year_list=[2021 + i for i in range(4)], country='US'
)

model_prophet = Prophet(daily_seasonality = True, holidays=us_holidays, changepoint_prior_scale=0.3,
                        holidays_prior_scale=1.0, n_changepoints=50, seasonality_prior_scale=0.01).fit(prophet_train_df_input)

future = model_prophet.make_future_dataframe(periods = fcast_time+3, freq = freq_option)
future = future[(future['ds'].dt.dayofweek < 5)&(~future['ds'].isin(us_holidays['ds']))] # 주말 제외 작업 & # 공휴일 제외 작업

df_forecast = model_prophet.predict(future)
# model_prophet.plot(df_forecast, xlabel = 'Date',ylabel ='price($)')
# model_prophet.plot_components(df_forecast);

[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
15:33:21 - cmdstanpy - INFO - Chain [1] start processing
15:33:22 - cmdstanpy - INFO - Chain [1] done processing


In [13]:
df_forecast.columns.to_list()

['ds',
 'trend',
 'yhat_lower',
 'yhat_upper',
 'trend_lower',
 'trend_upper',
 'Christmas Day',
 'Christmas Day_lower',
 'Christmas Day_upper',
 'Christmas Day (observed)',
 'Christmas Day (observed)_lower',
 'Christmas Day (observed)_upper',
 'Columbus Day',
 'Columbus Day_lower',
 'Columbus Day_upper',
 'Independence Day',
 'Independence Day_lower',
 'Independence Day_upper',
 'Independence Day (observed)',
 'Independence Day (observed)_lower',
 'Independence Day (observed)_upper',
 'Juneteenth National Independence Day',
 'Juneteenth National Independence Day_lower',
 'Juneteenth National Independence Day_upper',
 'Juneteenth National Independence Day (observed)',
 'Juneteenth National Independence Day (observed)_lower',
 'Juneteenth National Independence Day (observed)_upper',
 'Labor Day',
 'Labor Day_lower',
 'Labor Day_upper',
 'Martin Luther King Jr. Day',
 'Martin Luther King Jr. Day_lower',
 'Martin Luther King Jr. Day_upper',
 'Memorial Day',
 'Memorial Day_lower',
 'Memori

In [22]:
columns_to_drop = ['Christmas Day', 'Christmas Day_lower', 'Christmas Day_upper', 'Christmas Day (observed)', 'Christmas Day (observed)_lower', 
             'Christmas Day (observed)_upper', 'Columbus Day', 'Columbus Day_lower', 'Columbus Day_upper', 'Independence Day', 
             'Independence Day_lower', 'Independence Day_upper', 'Independence Day (observed)', 'Independence Day (observed)_lower', 
             'Independence Day (observed)_upper', 'Juneteenth National Independence Day', 'Juneteenth National Independence Day_lower', 
             'Juneteenth National Independence Day_upper', 'Juneteenth National Independence Day (observed)', 
             'Juneteenth National Independence Day (observed)_lower', 'Juneteenth National Independence Day (observed)_upper', 'Labor Day', 
             'Labor Day_lower', 'Labor Day_upper', 'Martin Luther King Jr. Day', 'Martin Luther King Jr. Day_lower', 
             'Martin Luther King Jr. Day_upper', 'Memorial Day', 'Memorial Day_lower', 'Memorial Day_upper', "New Year's Day", 
             "New Year's Day_lower", "New Year's Day_upper", "New Year's Day (observed)", "New Year's Day (observed)_lower", 
             "New Year's Day (observed)_upper", 'Thanksgiving', 'Thanksgiving_lower', 'Thanksgiving_upper', 'Veterans Day', 'Veterans Day_lower', 
             'Veterans Day_upper', 'Veterans Day (observed)', 'Veterans Day (observed)_lower', 'Veterans Day (observed)_upper',
             "Washington's Birthday", "Washington's Birthday_lower", "Washington's Birthday_upper"]

In [23]:
df_forecast.drop(columns_to_drop, axis=1, inplace=True)

In [18]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'stockdata' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
df_forecast.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


In [19]:
df_forecast.set_index('ds', inplace=True, drop=False)
fc = df_forecast[df_forecast['ds'].isin(df_recent['ds'])]['yhat']

mse = mean_squared_error(df_recent['y'], fc)
mae = mean_absolute_error(df_recent['y'], fc)
rmse = math.sqrt(mean_squared_error(df_recent['y'], fc))
mape = mean_absolute_percentage_error(df_recent['y'], fc)
r2score = r2_score(df_recent['y'],fc)
print(f'mse: {mse}, mae: {mae}, rmse: {rmse}, mape: {mape}, r2_score: {r2score}')

mse: 88.66646881345152, mae: 8.373797960338052, rmse: 9.416287421985988, mape: 0.045997049495457035, r2_score: -1.3514149805261004


In [ ]:
month_data = pd.read_csv('prophet한달치_예상값.csv')

In [40]:
month_data.set_index('ds',inplace=True,drop=False)

In [ ]:
year_data = df_forecast.iloc[:len(month_data)]

In [57]:
num = -5
display(month_data.iloc[[num]])
display(year_data.iloc[[num]])

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Christmas Day,Christmas Day_lower,Christmas Day_upper,Christmas Day (observed),...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
ds,,,,,,,,,,,,,,,,,,,,,
2024-01-25,2024-01-25,191.097148,188.723244,198.957197,188.107445,194.134054,0.0,0.0,0.0,0.0,...,-0.097753,-0.097753,-0.097753,2.90051,2.90051,2.90051,0.0,0.0,0.0,193.911653


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Christmas Day,Christmas Day_lower,Christmas Day_upper,Christmas Day (observed),...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
ds,,,,,,,,,,,,,,,,,,,,,
2024-01-25,2024-01-25,191.097148,188.080769,199.469944,187.864165,194.089467,0.0,0.0,0.0,0.0,...,-0.097753,-0.097753,-0.097753,2.90051,2.90051,2.90051,0.0,0.0,0.0,193.911653


In [26]:
df_forecast.to_csv('prophet_일년짜리예측.csv', index=False)

## union

In [27]:
# 아리마 예측 적재용
import pandas as pd
df_arima = pd.read_csv('arima_forecast.csv')# 'ARIMA_forecast_2024-01.csv')

df_arima['ds'] = pd.to_datetime(df_arima['ds'])

In [28]:
import pandas as pd
import yfinance as yf
start = '2021-01-01'
end = '2023-12-31'
symbol = 'AAPL' # "005930.KS"
df_merge = yf.download(symbol, start, end)
df_merge = df_merge.rename(columns={'Adj Close': 'Adj_Close'})
# df_merge['ds'] = pd.to_datetime(df_merge.index, format = '%Y-%m-%d') # 굳이 넣어야하나 고민중
# df_merge['y'] = df_merge['Close'] # 마찬가지로 굳이 넣어야하나 고민중
df_merge.head(3)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2021-01-04,133.520004,133.610001,126.760002,129.410004,127.002098,143301900
2021-01-05,128.889999,131.740005,128.429993,131.009995,128.572311,97664900
2021-01-06,127.720001,131.050003,126.379997,126.599998,124.244377,155088000


In [29]:
import numpy as np
from prophet import Prophet
df_merge['ds'] = pd.to_datetime(df_merge.index, format = '%Y-%m-%d') # 굳이 넣어야하나 고민중
df_merge['y'] = df_merge['Close'] # 마찬가지로 굳이 넣어야하나 고민중

In [30]:
df_merge.reset_index(drop=True, inplace=True)

In [31]:
df_close = df_merge[['ds','Close']]
df_close.head(3)

,ds,Close
0,2021-01-04,129.410004
1,2021-01-05,131.009995
2,2021-01-06,126.599998


In [32]:
df_prophet_concat=df_forecast[['ds','yhat','yhat_lower','yhat_upper']]
df_prophet_concat.head(3)

,ds,yhat,yhat_lower,yhat_upper
0,2021-01-04,132.273820,127.932490,136.725737
1,2021-01-05,132.082000,127.642462,136.570749
2,2021-01-06,131.791126,126.990223,136.446756


In [33]:
df_prophet_concat2 = pd.merge(df_prophet_concat, df_close, on='ds', how='left')
df_prophet_concat2.iloc[-252:-248]
df_prophet_concat2.info()
df_prophet_concat2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 996 entries, 0 to 995
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ds          996 non-null    datetime64[ns]
 1   yhat        996 non-null    float64       
 2   yhat_lower  996 non-null    float64       
 3   yhat_upper  996 non-null    float64       
 4   Close       745 non-null    float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 39.0 KB


,ds,yhat,yhat_lower,yhat_upper,Close
0,2021-01-04,132.273820,127.932490,136.725737,129.410004
1,2021-01-05,132.082000,127.642462,136.570749,131.009995
2,2021-01-06,131.791126,126.990223,136.446756,126.599998
3,2021-01-07,131.383895,126.508872,136.076737,130.919998
4,2021-01-08,131.252225,126.359464,135.403756,132.050003
...,...,...,...,...,...
991,2024-12-24,213.381351,-30.470782,458.666265,NaN
992,2024-12-26,212.872351,-33.800518,456.847381,NaN
993,2024-12-27,212.783411,-32.520627,458.933772,NaN
994,2024-12-30,212.122836,-34.515993,461.425211,NaN


In [34]:
df_arima_concat2 = pd.merge(df_arima, df_close, on='ds', how='left') # df_arima_concat 은 없는게 맞음 애초에 컬럼이 그것들 뿐이라 안만듦
df_arima_concat2.iloc[-22:-18]
df_arima_concat2

,ds,yhat,yhat_lower,yhat_upper,Close
0,2021-01-04,188.039993,181.956899,194.123088,129.410004
1,2021-01-05,129.410004,122.663076,136.156931,131.009995
2,2021-01-06,131.009995,124.264746,137.755243,126.599998
3,2021-01-07,126.599998,119.853352,133.346645,130.919998
4,2021-01-08,130.919998,124.172099,137.667897,132.050003
...,...,...,...,...,...
768,2024-01-24,195.179993,188.896540,201.463446,NaN
769,2024-01-25,194.500000,188.217844,200.782156,NaN
770,2024-01-26,194.169998,187.889184,200.450812,NaN
771,2024-01-29,192.419998,186.140136,198.699860,NaN


In [ ]:
df_arima_concat2.sort_values(by='ds',inplace=True)

In [35]:
df_prophet_concat2['model']='prophet'

In [36]:
df_arima_concat2['model']='arima'

In [37]:
combined_data = pd.concat([df_prophet_concat2, df_arima_concat2])
combined_data.reset_index(drop=True, inplace=True)

In [43]:
combined_data.iloc[950:1000]

,ds,yhat,yhat_lower,yhat_upper,Close,model
950,2024-10-24,196.432065,14.266490,381.805899,NaN,prophet
951,2024-10-25,196.732101,11.490261,385.946973,NaN,prophet
952,2024-10-28,197.298966,12.014562,387.027227,NaN,prophet
953,2024-10-29,197.603367,10.788621,387.693059,NaN,prophet
954,2024-10-30,197.804549,8.602951,388.586596,NaN,prophet
955,2024-10-31,197.884060,8.160420,391.966433,NaN,prophet
956,2024-11-01,198.232472,4.788578,393.137061,NaN,prophet
957,2024-11-04,199.079931,6.699851,392.422669,NaN,prophet
958,2024-11-05,199.520942,4.094406,394.612571,NaN,prophet
959,2024-11-06,199.877607,4.416749,398.194078,NaN,prophet


In [44]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'stockdata' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'combined_data' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
combined_data.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


In [3]:
import pandas as pd
import yfinance as yf
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from tqdm import tqdm

get_list={'samsung': '005930.KS', 'google': 'GOOG', 'microsoft': 'MSFT', 'amazon': 'AMZN', 'meta_facebook': 'META'}

for company, ticker in tqdm(get_list.items()):
    start = '2021-01-01'
    end = '2024-12-31'
    symbol = ticker # "005930.KS"
    df = yf.download(symbol, start, end)
    df = df.rename(columns={'Adj Close': 'Adj_Close'})
    # df['ds'] = pd.to_datetime(df.index, format = '%Y-%m-%d') # 굳이 넣어야하나 고민중
    # df['y'] = df['Close'] # 마찬가지로 굳이 넣어야하나 고민중
    display(df.head(3))


    # BigQuery 클라이언트 생성
    client = bigquery.Client()
    project_id = client.project #'actual-project-415811'

    ### 여기 설정 ###
    dataset_name = 'stockdata' # 여기 부분 원하는 데이터셋으로 설정
    ### 여기 설정 끝 ###

    # 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
    dataset_id = "{}.{}".format(project_id, dataset_name)

    # 데이터 세트 참조 생성
    dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

    try:
        # 데이터 세트가 존재하는지 확인
        client.get_dataset(dataset_ref)
        print("Dataset already exists.")
    except NotFound:
        # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
        dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
        dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
        dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
        print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

    # client = bigquery.Client()
    # dataset = 'stockdata' 

    ### 여기 설정 ###
    table_name = company+'_from2021-01-01' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
    ### 여기 설정 끝 ###

    table_id = "{}.{}".format(dataset_name,table_name)
    df.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2021-01-04,81000.0,84400.0,80200.0,83000.0,77922.695312,38655276
2021-01-05,81600.0,83900.0,81600.0,83900.0,78767.632812,35335669
2021-01-06,83300.0,84500.0,82100.0,82200.0,77171.640625,42089013


Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2021-01-04,87.876999,88.032501,85.392502,86.412003,86.412003,38038000
2021-01-05,86.250000,87.383499,85.900749,87.045998,87.045998,22906000
2021-01-06,85.131500,87.400002,84.949997,86.764503,86.764503,52042000


...

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2021-01-04,274.779999,275.000000,265.200012,268.940002,268.654968,15106100
2021-01-05,268.290009,272.399994,268.209991,270.970001,270.682800,9871600
2021-01-06,262.000000,267.750000,260.010010,263.309998,263.030914,24354100


Dataset already exists.


100%|██████████| 5/5 [00:38<00:00,  7.72s/it]


# 모의 주식 계산용 데이터 만들기

In [5]:
import pandas as pd
import yfinance as yf
start = '2021-01-01'
end = '2024-12-31'
symbol = 'AAPL' # "005930.KS"
df = yf.download(symbol, start, end)
df = df.rename(columns={'Adj Close': 'Adj_Close'})
# df['ds'] = pd.to_datetime(df.index, format = '%Y-%m-%d') # 굳이 넣어야하나 고민중
# df['y'] = df['Close'] # 마찬가지로 굳이 넣어야하나 고민중
df.head(3)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2021-01-04,133.520004,133.610001,126.760002,129.410004,127.002098,143301900
2021-01-05,128.889999,131.740005,128.429993,131.009995,128.572357,97664900
2021-01-06,127.720001,131.050003,126.379997,126.599998,124.244377,155088000


In [8]:
# df['datestamp'] = pd.to_datetime(df.index, format = '%Y-%m-%d')
df_real = df[['datestamp', 'Close']]
df_real = df_real.rename(columns={'Close': 'y'})

,datestamp,y
Date,,
2021-01-04,2021-01-04,129.410004
2021-01-05,2021-01-05,131.009995
2021-01-06,2021-01-06,126.599998
2021-01-07,2021-01-07,130.919998
2021-01-08,2021-01-08,132.050003
...,...,...
2024-03-04,2024-03-04,175.100006
2024-03-05,2024-03-05,170.119995
2024-03-06,2024-03-06,169.119995


In [9]:
df_real.reset_index(inplace=True, drop=True)

In [14]:
df_real[-1:]

,datestamp,y
799,2024-03-08,170.729996


In [11]:
df_predict = pd.read_csv('prophet_일년짜리예측.csv')

In [20]:
df_predict=df_predict[df_predict['ds']>'2024-03-08'][['ds','yhat']]

In [22]:
df_predict.reset_index(inplace=True, drop=True)

In [24]:
df_predict.rename(columns={'ds': 'datestamp', 'yhat': 'y'}, inplace=True)

In [25]:
df_predict

,datestamp,y
0,2024-03-11,186.128756
1,2024-03-12,186.552684
2,2024-03-13,186.911373
3,2024-03-14,187.176142
4,2024-03-15,187.727263
...,...,...
199,2024-12-24,213.381351
200,2024-12-26,212.872351
201,2024-12-27,212.783411
202,2024-12-30,212.122836


In [29]:
df_real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   datestamp  800 non-null    datetime64[ns]
 1   y          800 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 12.6 KB


In [31]:
df_predict['datestamp']= pd.to_datetime(df_predict['datestamp'])

In [32]:
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   datestamp  204 non-null    datetime64[ns]
 1   y          204 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.3 KB


In [34]:
calculator_data=pd.concat([df_real, df_predict])

In [35]:
calculator_data

,datestamp,y
0,2021-01-04,129.410004
1,2021-01-05,131.009995
2,2021-01-06,126.599998
3,2021-01-07,130.919998
4,2021-01-08,132.050003
...,...,...
199,2024-12-24,213.381351
200,2024-12-26,212.872351
201,2024-12-27,212.783411
202,2024-12-30,212.122836


In [36]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'stockdata' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'calculator_data' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
calculator_data.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


In [ ]:
# 아리마 예측 적재용
import pandas as pd
df_arima = pd.read_csv('arima_forecast.csv')# 'ARIMA_forecast_2024-01.csv')

df_arima['ds'] = pd.to_datetime(df_arima['ds'])

In [ ]:
import pandas as pd
import yfinance as yf
start = '2021-01-01'
end = '2023-12-31'
symbol = 'AAPL' # "005930.KS"
df_merge = yf.download(symbol, start, end)
df_merge = df_merge.rename(columns={'Adj Close': 'Adj_Close'})
# df_merge['ds'] = pd.to_datetime(df_merge.index, format = '%Y-%m-%d') # 굳이 넣어야하나 고민중
# df_merge['y'] = df_merge['Close'] # 마찬가지로 굳이 넣어야하나 고민중
df_merge.head(3)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2021-01-04,133.520004,133.610001,126.760002,129.410004,127.002098,143301900
2021-01-05,128.889999,131.740005,128.429993,131.009995,128.572311,97664900
2021-01-06,127.720001,131.050003,126.379997,126.599998,124.244377,155088000


In [ ]:
import numpy as np
from prophet import Prophet
df_merge['ds'] = pd.to_datetime(df_merge.index, format = '%Y-%m-%d') # 굳이 넣어야하나 고민중
df_merge['y'] = df_merge['Close'] # 마찬가지로 굳이 넣어야하나 고민중

In [ ]:
df_merge.reset_index(drop=True, inplace=True)

In [ ]:
df_close = df_merge[['ds','Close']]
df_close.head(3)

,ds,Close
0,2021-01-04,129.410004
1,2021-01-05,131.009995
2,2021-01-06,126.599998


In [ ]:
df_prophet_concat=df_forecast[['ds','yhat','yhat_lower','yhat_upper']]
df_prophet_concat.head(3)

,ds,yhat,yhat_lower,yhat_upper
0,2021-01-04,132.273820,127.932490,136.725737
1,2021-01-05,132.082000,127.642462,136.570749
2,2021-01-06,131.791126,126.990223,136.446756


In [ ]:
df_prophet_concat2 = pd.merge(df_prophet_concat, df_close, on='ds', how='left')
df_prophet_concat2.iloc[-252:-248]
df_prophet_concat2.info()
df_prophet_concat2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 996 entries, 0 to 995
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ds          996 non-null    datetime64[ns]
 1   yhat        996 non-null    float64       
 2   yhat_lower  996 non-null    float64       
 3   yhat_upper  996 non-null    float64       
 4   Close       745 non-null    float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 39.0 KB


,ds,yhat,yhat_lower,yhat_upper,Close
0,2021-01-04,132.273820,127.932490,136.725737,129.410004
1,2021-01-05,132.082000,127.642462,136.570749,131.009995
2,2021-01-06,131.791126,126.990223,136.446756,126.599998
3,2021-01-07,131.383895,126.508872,136.076737,130.919998
4,2021-01-08,131.252225,126.359464,135.403756,132.050003
...,...,...,...,...,...
991,2024-12-24,213.381351,-30.470782,458.666265,NaN
992,2024-12-26,212.872351,-33.800518,456.847381,NaN
993,2024-12-27,212.783411,-32.520627,458.933772,NaN
994,2024-12-30,212.122836,-34.515993,461.425211,NaN


In [ ]:
df_arima_concat2 = pd.merge(df_arima, df_close, on='ds', how='left') # df_arima_concat 은 없는게 맞음 애초에 컬럼이 그것들 뿐이라 안만듦
df_arima_concat2.iloc[-22:-18]
df_arima_concat2

,ds,yhat,yhat_lower,yhat_upper,Close
0,2021-01-04,188.039993,181.956899,194.123088,129.410004
1,2021-01-05,129.410004,122.663076,136.156931,131.009995
2,2021-01-06,131.009995,124.264746,137.755243,126.599998
3,2021-01-07,126.599998,119.853352,133.346645,130.919998
4,2021-01-08,130.919998,124.172099,137.667897,132.050003
...,...,...,...,...,...
768,2024-01-24,195.179993,188.896540,201.463446,NaN
769,2024-01-25,194.500000,188.217844,200.782156,NaN
770,2024-01-26,194.169998,187.889184,200.450812,NaN
771,2024-01-29,192.419998,186.140136,198.699860,NaN


In [ ]:
df_arima_concat2.sort_values(by='ds',inplace=True)

In [ ]:
df_prophet_concat2['model']='prophet'

In [ ]:
df_arima_concat2['model']='arima'

In [ ]:
combined_data = pd.concat([df_prophet_concat2, df_arima_concat2])
combined_data.reset_index(drop=True, inplace=True)

In [ ]:
combined_data.iloc[950:1000]

,ds,yhat,yhat_lower,yhat_upper,Close,model
950,2024-10-24,196.432065,14.266490,381.805899,NaN,prophet
951,2024-10-25,196.732101,11.490261,385.946973,NaN,prophet
952,2024-10-28,197.298966,12.014562,387.027227,NaN,prophet
953,2024-10-29,197.603367,10.788621,387.693059,NaN,prophet
954,2024-10-30,197.804549,8.602951,388.586596,NaN,prophet
955,2024-10-31,197.884060,8.160420,391.966433,NaN,prophet
956,2024-11-01,198.232472,4.788578,393.137061,NaN,prophet
957,2024-11-04,199.079931,6.699851,392.422669,NaN,prophet
958,2024-11-05,199.520942,4.094406,394.612571,NaN,prophet
959,2024-11-06,199.877607,4.416749,398.194078,NaN,prophet


In [ ]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'stockdata' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'combined_data' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
combined_data.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


In [ ]:
import pandas as pd
import yfinance as yf
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from tqdm import tqdm

get_list={'samsung': '005930.KS', 'google': 'GOOG', 'microsoft': 'MSFT', 'amazon': 'AMZN', 'meta_facebook': 'META'}

for company, ticker in tqdm(get_list.items()):
    start = '2021-01-01'
    end = '2024-12-31'
    symbol = ticker # "005930.KS"
    df = yf.download(symbol, start, end)
    df = df.rename(columns={'Adj Close': 'Adj_Close'})
    # df['ds'] = pd.to_datetime(df.index, format = '%Y-%m-%d') # 굳이 넣어야하나 고민중
    # df['y'] = df['Close'] # 마찬가지로 굳이 넣어야하나 고민중
    display(df.head(3))


    # BigQuery 클라이언트 생성
    client = bigquery.Client()
    project_id = client.project #'actual-project-415811'

    ### 여기 설정 ###
    dataset_name = 'stockdata' # 여기 부분 원하는 데이터셋으로 설정
    ### 여기 설정 끝 ###

    # 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
    dataset_id = "{}.{}".format(project_id, dataset_name)

    # 데이터 세트 참조 생성
    dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

    try:
        # 데이터 세트가 존재하는지 확인
        client.get_dataset(dataset_ref)
        print("Dataset already exists.")
    except NotFound:
        # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
        dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
        dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
        dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
        print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

    # client = bigquery.Client()
    # dataset = 'stockdata' 

    ### 여기 설정 ###
    table_name = company+'_from2021-01-01' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
    ### 여기 설정 끝 ###

    table_id = "{}.{}".format(dataset_name,table_name)
    df.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2021-01-04,81000.0,84400.0,80200.0,83000.0,77922.695312,38655276
2021-01-05,81600.0,83900.0,81600.0,83900.0,78767.632812,35335669
2021-01-06,83300.0,84500.0,82100.0,82200.0,77171.640625,42089013


Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2021-01-04,87.876999,88.032501,85.392502,86.412003,86.412003,38038000
2021-01-05,86.250000,87.383499,85.900749,87.045998,87.045998,22906000
2021-01-06,85.131500,87.400002,84.949997,86.764503,86.764503,52042000


...

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2021-01-04,274.779999,275.000000,265.200012,268.940002,268.654968,15106100
2021-01-05,268.290009,272.399994,268.209991,270.970001,270.682800,9871600
2021-01-06,262.000000,267.750000,260.010010,263.309998,263.030914,24354100


Dataset already exists.


100%|██████████| 5/5 [00:38<00:00,  7.72s/it]


# 모의 주식 계산용 데이터 만들기

In [ ]:
import pandas as pd
import yfinance as yf
start = '2021-01-01'
end = '2024-12-31'
symbol = 'AAPL' # "005930.KS"
df = yf.download(symbol, start, end)
df = df.rename(columns={'Adj Close': 'Adj_Close'})
# df['ds'] = pd.to_datetime(df.index, format = '%Y-%m-%d') # 굳이 넣어야하나 고민중
# df['y'] = df['Close'] # 마찬가지로 굳이 넣어야하나 고민중
df.head(3)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2021-01-04,133.520004,133.610001,126.760002,129.410004,127.002098,143301900
2021-01-05,128.889999,131.740005,128.429993,131.009995,128.572357,97664900
2021-01-06,127.720001,131.050003,126.379997,126.599998,124.244377,155088000


In [ ]:
# df['datestamp'] = pd.to_datetime(df.index, format = '%Y-%m-%d')
df_real = df[['datestamp', 'Close']]
df_real = df_real.rename(columns={'Close': 'y'})

,datestamp,y
Date,,
2021-01-04,2021-01-04,129.410004
2021-01-05,2021-01-05,131.009995
2021-01-06,2021-01-06,126.599998
2021-01-07,2021-01-07,130.919998
2021-01-08,2021-01-08,132.050003
...,...,...
2024-03-04,2024-03-04,175.100006
2024-03-05,2024-03-05,170.119995
2024-03-06,2024-03-06,169.119995


In [ ]:
df_real.reset_index(inplace=True, drop=True)

In [ ]:
df_real[-1:]

,datestamp,y
799,2024-03-08,170.729996


In [ ]:
df_predict = pd.read_csv('prophet_일년짜리예측.csv')

In [ ]:
df_predict=df_predict[df_predict['ds']>'2024-03-08'][['ds','yhat']]

In [ ]:
df_predict.reset_index(inplace=True, drop=True)

In [ ]:
df_predict.rename(columns={'ds': 'datestamp', 'yhat': 'y'}, inplace=True)

In [ ]:
df_predict

,datestamp,y
0,2024-03-11,186.128756
1,2024-03-12,186.552684
2,2024-03-13,186.911373
3,2024-03-14,187.176142
4,2024-03-15,187.727263
...,...,...
199,2024-12-24,213.381351
200,2024-12-26,212.872351
201,2024-12-27,212.783411
202,2024-12-30,212.122836


In [ ]:
df_real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   datestamp  800 non-null    datetime64[ns]
 1   y          800 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 12.6 KB


In [ ]:
df_predict['datestamp']= pd.to_datetime(df_predict['datestamp'])

In [ ]:
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   datestamp  204 non-null    datetime64[ns]
 1   y          204 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.3 KB


In [ ]:
calculator_data=pd.concat([df_real, df_predict])

In [ ]:
calculator_data

,datestamp,y
0,2021-01-04,129.410004
1,2021-01-05,131.009995
2,2021-01-06,126.599998
3,2021-01-07,130.919998
4,2021-01-08,132.050003
...,...,...
199,2024-12-24,213.381351
200,2024-12-26,212.872351
201,2024-12-27,212.783411
202,2024-12-30,212.122836


In [ ]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'stockdata' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'calculator_data' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
calculator_data.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


In [2]:
import pandas as pd
hom = pd.DataFrame({
    'title': ['홈'],
    'url': ['https://finance.yahoo.com/quote/AAPL?.tsrc=fin-srch']
})

In [4]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'major_indices' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'article_home' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
hom.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


# 긴급 더 좋은 제대로된듯한 파라미터 발견해서 데이터 교체

In [8]:
start = '2021-01-01' # '2022-01-01'
end = '2023-12-31'
symbol = 'AAPL' # "005930.KS"
prophet_train_df = yf.download(symbol, start, end)
prophet_train_df = prophet_train_df.rename(columns={'Adj Close': 'Adj_Close'})
prophet_train_df['ds'] = pd.to_datetime(prophet_train_df.index, format = '%Y-%m-%d')
prophet_train_df['y'] = prophet_train_df['Close']

prophet_train_df_input = prophet_train_df[['ds','y']]

fcast_time = 365 #365 # 365일 예측
freq_option = 'D'


# 문자열을 datetime 객체로 변환
end_date = datetime.strptime(end, '%Y-%m-%d')
start_date_r = end_date + timedelta(days=1) # fcast_time+1 만큼의 일수를 더함
end_date_r = end_date + timedelta(days=fcast_time+2) # fcast_time+1 만큼의 일수를 더함 end날 이전까지 가져와서 +1 해줌
start_r = start_date_r.strftime('%Y-%m-%d') # 문자로 바꿔줌
end_r = end_date_r.strftime('%Y-%m-%d') # 문자로 바꿔줌
ticker_r = 'AAPL'

df_recent = yf.download(ticker_r, start=start_r, end=end_r)
df_recent['ds'] = pd.to_datetime(df_recent.index, format = '%Y-%m-%d')
df_recent['y'] = df_recent['Close']

us_holidays = make_holidays_df(
    year_list=[2021 + i for i in range(4)], country='US'
)

model_prophet = Prophet(holidays=us_holidays, changepoint_prior_scale=0.3,holidays_prior_scale=10, 
                        n_changepoints=50, seasonality_mode='multiplicative', seasonality_prior_scale=0.01).fit(prophet_train_df)

future = model_prophet.make_future_dataframe(periods = fcast_time+3, freq = freq_option)
future = future[(future['ds'].dt.dayofweek < 5)&(~future['ds'].isin(us_holidays['ds']))] # 주말 제외 작업 & # 공휴일 제외 작업

df_forecast = model_prophet.predict(future)
# model_prophet.plot(df_forecast, xlabel = 'Date',ylabel ='price($)')
# model_prophet.plot_components(df_forecast);

[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
00:51:05 - cmdstanpy - INFO - Chain [1] start processing
00:51:05 - cmdstanpy - INFO - Chain [1] done processing


In [10]:
columns_to_drop = ['Christmas Day', 'Christmas Day_lower', 'Christmas Day_upper', 'Christmas Day (observed)', 'Christmas Day (observed)_lower', 
             'Christmas Day (observed)_upper', 'Columbus Day', 'Columbus Day_lower', 'Columbus Day_upper', 'Independence Day', 
             'Independence Day_lower', 'Independence Day_upper', 'Independence Day (observed)', 'Independence Day (observed)_lower', 
             'Independence Day (observed)_upper', 'Juneteenth National Independence Day', 'Juneteenth National Independence Day_lower', 
             'Juneteenth National Independence Day_upper', 'Juneteenth National Independence Day (observed)', 
             'Juneteenth National Independence Day (observed)_lower', 'Juneteenth National Independence Day (observed)_upper', 'Labor Day', 
             'Labor Day_lower', 'Labor Day_upper', 'Martin Luther King Jr. Day', 'Martin Luther King Jr. Day_lower', 
             'Martin Luther King Jr. Day_upper', 'Memorial Day', 'Memorial Day_lower', 'Memorial Day_upper', "New Year's Day", 
             "New Year's Day_lower", "New Year's Day_upper", "New Year's Day (observed)", "New Year's Day (observed)_lower", 
             "New Year's Day (observed)_upper", 'Thanksgiving', 'Thanksgiving_lower', 'Thanksgiving_upper', 'Veterans Day', 'Veterans Day_lower', 
             'Veterans Day_upper', 'Veterans Day (observed)', 'Veterans Day (observed)_lower', 'Veterans Day (observed)_upper',
             "Washington's Birthday", "Washington's Birthday_lower", "Washington's Birthday_upper"]

df_forecast.drop(columns_to_drop, axis=1, inplace=True)

from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'stockdata' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
df_forecast.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


In [11]:
df_forecast.to_csv('prophet_일년짜리예측2.csv', index=False)

# 예측 데이터 아리마와 같이 합치기

In [12]:
# 아리마 예측 적재용
import pandas as pd
df_arima = pd.read_csv('arima_forecast.csv')# 'ARIMA_forecast_2024-01.csv')

df_arima['ds'] = pd.to_datetime(df_arima['ds'])

In [13]:
import pandas as pd
import yfinance as yf
start = '2021-01-01'
end = '2023-12-31'
symbol = 'AAPL' # "005930.KS"
df_merge = yf.download(symbol, start, end)
df_merge = df_merge.rename(columns={'Adj Close': 'Adj_Close'})
# df_merge['ds'] = pd.to_datetime(df_merge.index, format = '%Y-%m-%d') # 굳이 넣어야하나 고민중
# df_merge['y'] = df_merge['Close'] # 마찬가지로 굳이 넣어야하나 고민중
df_merge.head(3)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2021-01-04,133.520004,133.610001,126.760002,129.410004,127.002098,143301900
2021-01-05,128.889999,131.740005,128.429993,131.009995,128.572342,97664900
2021-01-06,127.720001,131.050003,126.379997,126.599998,124.244385,155088000


In [14]:
import numpy as np
from prophet import Prophet
df_merge['ds'] = pd.to_datetime(df_merge.index, format = '%Y-%m-%d') # 굳이 넣어야하나 고민중
df_merge['y'] = df_merge['Close'] # 마찬가지로 굳이 넣어야하나 고민중

In [15]:
df_merge.reset_index(drop=True, inplace=True)

In [16]:
df_close = df_merge[['ds','Close']]
df_close.head(3)

,ds,Close
0,2021-01-04,129.410004
1,2021-01-05,131.009995
2,2021-01-06,126.599998


In [17]:
df_prophet_concat=df_forecast[['ds','yhat','yhat_lower','yhat_upper']]
df_prophet_concat.head(3)

,ds,yhat,yhat_lower,yhat_upper
0,2021-01-04,132.363156,127.838944,136.780391
1,2021-01-05,132.229096,127.872543,136.915568
2,2021-01-06,131.995442,127.949439,136.527183


In [18]:
df_prophet_concat2 = pd.merge(df_prophet_concat, df_close, on='ds', how='left')
df_prophet_concat2.iloc[-252:-248]
df_prophet_concat2.info()
df_prophet_concat2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 996 entries, 0 to 995
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ds          996 non-null    datetime64[ns]
 1   yhat        996 non-null    float64       
 2   yhat_lower  996 non-null    float64       
 3   yhat_upper  996 non-null    float64       
 4   Close       745 non-null    float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 39.0 KB


,ds,yhat,yhat_lower,yhat_upper,Close
0,2021-01-04,132.363156,127.838944,136.780391,129.410004
1,2021-01-05,132.229096,127.872543,136.915568,131.009995
2,2021-01-06,131.995442,127.949439,136.527183,126.599998
3,2021-01-07,131.643013,127.022322,135.892379,130.919998
4,2021-01-08,131.508340,126.839938,135.899648,132.050003
...,...,...,...,...,...
991,2024-12-24,209.802502,-37.895094,461.036431,NaN
992,2024-12-26,209.109973,-41.273523,460.706581,NaN
993,2024-12-27,208.927639,-40.578301,465.207966,NaN
994,2024-12-30,207.864764,-42.002588,461.016105,NaN


In [19]:
df_arima_concat2 = pd.merge(df_arima, df_close, on='ds', how='left') # df_arima_concat 은 없는게 맞음 애초에 컬럼이 그것들 뿐이라 안만듦
df_arima_concat2.iloc[-22:-18]
df_arima_concat2

,ds,yhat,yhat_lower,yhat_upper,Close
0,2021-01-04,188.039993,181.956899,194.123088,129.410004
1,2021-01-05,129.410004,122.663076,136.156931,131.009995
2,2021-01-06,131.009995,124.264746,137.755243,126.599998
3,2021-01-07,126.599998,119.853352,133.346645,130.919998
4,2021-01-08,130.919998,124.172099,137.667897,132.050003
...,...,...,...,...,...
768,2024-01-24,195.179993,188.896540,201.463446,NaN
769,2024-01-25,194.500000,188.217844,200.782156,NaN
770,2024-01-26,194.169998,187.889184,200.450812,NaN
771,2024-01-29,192.419998,186.140136,198.699860,NaN


In [20]:
df_arima_concat2.sort_values(by='ds',inplace=True)

In [21]:
df_prophet_concat2['model']='prophet'

In [22]:
df_arima_concat2['model']='arima'

In [23]:
combined_data = pd.concat([df_prophet_concat2, df_arima_concat2])
combined_data.reset_index(drop=True, inplace=True)

In [24]:
combined_data.iloc[950:1000]

,ds,yhat,yhat_lower,yhat_upper,Close,model
950,2024-10-24,190.812117,13.023401,361.153137,NaN,prophet
951,2024-10-25,191.108853,13.841496,361.331476,NaN,prophet
952,2024-10-28,191.691209,12.302634,361.670156,NaN,prophet
953,2024-10-29,192.083353,8.434298,363.868079,NaN,prophet
954,2024-10-30,192.329955,6.558121,366.426514,NaN,prophet
955,2024-10-31,192.397878,6.184173,370.078392,NaN,prophet
956,2024-11-01,192.798323,5.091495,372.153187,NaN,prophet
957,2024-11-04,193.837178,-0.422352,373.045762,NaN,prophet
958,2024-11-05,194.423854,3.487495,376.746185,NaN,prophet
959,2024-11-06,194.880884,0.252835,378.358109,NaN,prophet


In [25]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'stockdata' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'combined_data' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
combined_data.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


# 모의계산기용

In [26]:
import pandas as pd
import yfinance as yf
start = '2021-01-01'
end = '2024-12-31'
symbol = 'AAPL' # "005930.KS"
df = yf.download(symbol, start, end)
df = df.rename(columns={'Adj Close': 'Adj_Close'})
# df['ds'] = pd.to_datetime(df.index, format = '%Y-%m-%d') # 굳이 넣어야하나 고민중
# df['y'] = df['Close'] # 마찬가지로 굳이 넣어야하나 고민중
df.head(3)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2021-01-04,133.520004,133.610001,126.760002,129.410004,127.002121,143301900
2021-01-05,128.889999,131.740005,128.429993,131.009995,128.572311,97664900
2021-01-06,127.720001,131.050003,126.379997,126.599998,124.244385,155088000


In [28]:
df['datestamp'] = pd.to_datetime(df.index, format = '%Y-%m-%d')
df_real = df[['datestamp', 'Close']]
df_real = df_real.rename(columns={'Close': 'y'})

In [29]:
df_real.reset_index(inplace=True, drop=True)

In [30]:
df_real[-1:]

,datestamp,y
799,2024-03-08,170.729996


In [31]:
df_predict = pd.read_csv('prophet_일년짜리예측2.csv')

In [32]:
df_predict=df_predict[df_predict['ds']>'2024-03-08'][['ds','yhat']]

In [33]:
df_predict.reset_index(inplace=True, drop=True)

In [34]:
df_predict.rename(columns={'ds': 'datestamp', 'yhat': 'y'}, inplace=True)

In [35]:
df_predict

,datestamp,y
0,2024-03-11,181.534999
1,2024-03-12,182.115842
2,2024-03-13,182.603286
3,2024-03-14,182.953081
4,2024-03-15,183.639336
...,...,...
199,2024-12-24,209.802502
200,2024-12-26,209.109973
201,2024-12-27,208.927639
202,2024-12-30,207.864764


In [36]:
df_real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   datestamp  800 non-null    datetime64[ns]
 1   y          800 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 12.6 KB


In [37]:
df_predict['datestamp']= pd.to_datetime(df_predict['datestamp'])

In [38]:
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   datestamp  204 non-null    datetime64[ns]
 1   y          204 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.3 KB


In [39]:
calculator_data=pd.concat([df_real, df_predict])

In [40]:
calculator_data

,datestamp,y
0,2021-01-04,129.410004
1,2021-01-05,131.009995
2,2021-01-06,126.599998
3,2021-01-07,130.919998
4,2021-01-08,132.050003
...,...,...
199,2024-12-24,209.802502
200,2024-12-26,209.109973
201,2024-12-27,208.927639
202,2024-12-30,207.864764


In [41]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'stockdata' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'calculator_data' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
calculator_data.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


# 예측 평가 지표

In [42]:
metrics = pd.read_csv('metrics_prophet(final)2.csv')

In [43]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'predict_metrics' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'metrics_prophet_aapl' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
metrics.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


In [44]:
# 아리마 예측 적재용
import pandas as pd
metrics_arima = pd.read_csv('metrics_arima.csv')

In [45]:
# metrics_prophet = metrics
metrics_prophet =pd.read_csv('metrics_prophet(final)2.csv')

In [46]:
metrics_prophet = metrics_prophet.assign(model='prophet')
metrics_arima = metrics_arima.assign(model='arima')

In [47]:
metrics_combined = pd.concat([metrics_prophet,metrics_arima])

In [48]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'predict_metrics' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'metrics_combined_aapl' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
metrics_combined.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


# 긴급 더 좋은 제대로된듯한 파라미터 발견해서 데이터 교체 holidays window값 넣기

In [23]:
start = '2021-01-01' # '2022-01-01'
end = '2023-12-31'
symbol = 'AAPL' # "005930.KS"
total_train_df = yf.download(symbol, start, end)
total_train_df = total_train_df.rename(columns={'Adj Close': 'Adj_Close'})
total_train_df['ds'] = pd.to_datetime(total_train_df.index, format = '%Y-%m-%d')
total_train_df['y'] = total_train_df['Close']

total_train_df_input = total_train_df[['ds','y']]

fcast_time = 365 # 365일 예측
freq_option = 'D'


# 문자열을 datetime 객체로 변환
end_date = datetime.strptime(end, '%Y-%m-%d')
start_date_r = end_date + timedelta(days=1) # fcast_time+1 만큼의 일수를 더함
end_date_r = end_date + timedelta(days=fcast_time+2) # fcast_time+1 만큼의 일수를 더함 end날 이전까지 가져와서 +1 해줌
start_r = start_date_r.strftime('%Y-%m-%d') # 문자로 바꿔줌
end_r = end_date_r.strftime('%Y-%m-%d') # 문자로 바꿔줌
ticker_r = 'AAPL'

df_recent = yf.download(ticker_r, start=start_r, end=end_r)
df_recent['ds'] = pd.to_datetime(df_recent.index, format = '%Y-%m-%d')
df_recent['y'] = df_recent['Close']

us_holidays = make_holidays_df(
    year_list=[2021 + i for i in range(4)], country='US'
)
us_holidays = us_holidays.assign(lower_window=-1, upper_window=1)
total_train_df_input = total_train_df[['ds','y']]
model_prophet = Prophet(holidays=us_holidays, changepoint_prior_scale=0.3,
                        holidays_prior_scale=10.0, n_changepoints=50, seasonality_mode='multiplicative', seasonality_prior_scale=0.01).fit(total_train_df_input)

future = model_prophet.make_future_dataframe(periods = fcast_time+3, freq = freq_option)
future = future[(future['ds'].dt.dayofweek < 5)&(~future['ds'].isin(us_holidays['ds']))] # 주말 제외 작업 & # 공휴일 제외 작업

df_forecast = model_prophet.predict(future)
# model_prophet.plot(df_forecast, xlabel = 'Date',ylabel ='price($)')
# model_prophet.plot_components(df_forecast);

[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
18:36:50 - cmdstanpy - INFO - Chain [1] start processing
18:36:50 - cmdstanpy - INFO - Chain [1] done processing


In [24]:
df_forecast.to_csv('prophet_일년짜리예측3.csv', index=False)

In [25]:
columns_to_drop = ['Christmas Day', 'Christmas Day_lower', 'Christmas Day_upper', 'Christmas Day (observed)', 'Christmas Day (observed)_lower', 
             'Christmas Day (observed)_upper', 'Columbus Day', 'Columbus Day_lower', 'Columbus Day_upper', 'Independence Day', 
             'Independence Day_lower', 'Independence Day_upper', 'Independence Day (observed)', 'Independence Day (observed)_lower', 
             'Independence Day (observed)_upper', 'Juneteenth National Independence Day', 'Juneteenth National Independence Day_lower', 
             'Juneteenth National Independence Day_upper', 'Juneteenth National Independence Day (observed)', 
             'Juneteenth National Independence Day (observed)_lower', 'Juneteenth National Independence Day (observed)_upper', 'Labor Day', 
             'Labor Day_lower', 'Labor Day_upper', 'Martin Luther King Jr. Day', 'Martin Luther King Jr. Day_lower', 
             'Martin Luther King Jr. Day_upper', 'Memorial Day', 'Memorial Day_lower', 'Memorial Day_upper', "New Year's Day", 
             "New Year's Day_lower", "New Year's Day_upper", "New Year's Day (observed)", "New Year's Day (observed)_lower", 
             "New Year's Day (observed)_upper", 'Thanksgiving', 'Thanksgiving_lower', 'Thanksgiving_upper', 'Veterans Day', 'Veterans Day_lower', 
             'Veterans Day_upper', 'Veterans Day (observed)', 'Veterans Day (observed)_lower', 'Veterans Day (observed)_upper',
             "Washington's Birthday", "Washington's Birthday_lower", "Washington's Birthday_upper"]

df_forecast.drop(columns_to_drop, axis=1, inplace=True)

from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'stockdata' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
df_forecast.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


# 예측 데이터 아리마와 같이 합치기

In [26]:
# 아리마 예측 적재용
import pandas as pd
df_arima = pd.read_csv('arima_forecast.csv')# 'ARIMA_forecast_2024-01.csv')

df_arima['ds'] = pd.to_datetime(df_arima['ds'])

In [27]:
import pandas as pd
import yfinance as yf
start = '2021-01-01'
end = '2023-12-31'
symbol = 'AAPL' # "005930.KS"
df_merge = yf.download(symbol, start, end)
df_merge = df_merge.rename(columns={'Adj Close': 'Adj_Close'})
# df_merge['ds'] = pd.to_datetime(df_merge.index, format = '%Y-%m-%d') # 굳이 넣어야하나 고민중
# df_merge['y'] = df_merge['Close'] # 마찬가지로 굳이 넣어야하나 고민중
df_merge.head(3)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2021-01-04,133.520004,133.610001,126.760002,129.410004,127.002106,143301900
2021-01-05,128.889999,131.740005,128.429993,131.009995,128.572311,97664900
2021-01-06,127.720001,131.050003,126.379997,126.599998,124.244385,155088000


In [28]:
import numpy as np
from prophet import Prophet
df_merge['ds'] = pd.to_datetime(df_merge.index, format = '%Y-%m-%d') # 굳이 넣어야하나 고민중
df_merge['y'] = df_merge['Close'] # 마찬가지로 굳이 넣어야하나 고민중

In [29]:
df_merge.reset_index(drop=True, inplace=True)

In [30]:
df_close = df_merge[['ds','Close']]
df_close.head(3)

,ds,Close
0,2021-01-04,129.410004
1,2021-01-05,131.009995
2,2021-01-06,126.599998


In [31]:
df_prophet_concat=df_forecast[['ds','yhat','yhat_lower','yhat_upper']]
df_prophet_concat.head(3)

,ds,yhat,yhat_lower,yhat_upper
0,2021-01-04,132.499399,128.127260,137.090452
1,2021-01-05,132.408105,127.963075,136.961630
2,2021-01-06,132.118526,127.517879,136.732033


In [32]:
df_prophet_concat2 = pd.merge(df_prophet_concat, df_close, on='ds', how='left')
df_prophet_concat2.iloc[-252:-248]
df_prophet_concat2.info()
df_prophet_concat2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 996 entries, 0 to 995
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ds          996 non-null    datetime64[ns]
 1   yhat        996 non-null    float64       
 2   yhat_lower  996 non-null    float64       
 3   yhat_upper  996 non-null    float64       
 4   Close       745 non-null    float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 39.0 KB


,ds,yhat,yhat_lower,yhat_upper,Close
0,2021-01-04,132.499399,128.127260,137.090452,129.410004
1,2021-01-05,132.408105,127.963075,136.961630,131.009995
2,2021-01-06,132.118526,127.517879,136.732033,126.599998
3,2021-01-07,131.759670,127.632702,136.441782,130.919998
4,2021-01-08,131.704658,127.240134,136.128164,132.050003
...,...,...,...,...,...
991,2024-12-24,210.580028,-33.020901,444.445939,NaN
992,2024-12-26,208.749040,-35.231140,442.180967,NaN
993,2024-12-27,209.741494,-38.388024,447.056764,NaN
994,2024-12-30,208.554550,-35.785146,447.167327,NaN


In [33]:
df_arima_concat2 = pd.merge(df_arima, df_close, on='ds', how='left') # df_arima_concat 은 없는게 맞음 애초에 컬럼이 그것들 뿐이라 안만듦
df_arima_concat2.iloc[-22:-18]
df_arima_concat2

,ds,yhat,yhat_lower,yhat_upper,Close
0,2021-01-04,188.039993,181.956899,194.123088,129.410004
1,2021-01-05,129.410004,122.663076,136.156931,131.009995
2,2021-01-06,131.009995,124.264746,137.755243,126.599998
3,2021-01-07,126.599998,119.853352,133.346645,130.919998
4,2021-01-08,130.919998,124.172099,137.667897,132.050003
...,...,...,...,...,...
768,2024-01-24,195.179993,188.896540,201.463446,NaN
769,2024-01-25,194.500000,188.217844,200.782156,NaN
770,2024-01-26,194.169998,187.889184,200.450812,NaN
771,2024-01-29,192.419998,186.140136,198.699860,NaN


In [34]:
df_arima_concat2.sort_values(by='ds',inplace=True)

In [35]:
df_prophet_concat2['model']='prophet'

In [36]:
df_arima_concat2['model']='arima'

In [37]:
combined_data = pd.concat([df_prophet_concat2, df_arima_concat2])
combined_data.reset_index(drop=True, inplace=True)

In [39]:
combined_data

,ds,yhat,yhat_lower,yhat_upper,Close,model
0,2021-01-04,132.499399,128.127260,137.090452,129.410004,prophet
1,2021-01-05,132.408105,127.963075,136.961630,131.009995,prophet
2,2021-01-06,132.118526,127.517879,136.732033,126.599998,prophet
3,2021-01-07,131.759670,127.632702,136.441782,130.919998,prophet
4,2021-01-08,131.704658,127.240134,136.128164,132.050003,prophet
...,...,...,...,...,...,...
1764,2024-01-24,195.179993,188.896540,201.463446,NaN,arima
1765,2024-01-25,194.500000,188.217844,200.782156,NaN,arima
1766,2024-01-26,194.169998,187.889184,200.450812,NaN,arima
1767,2024-01-29,192.419998,186.140136,198.699860,NaN,arima


In [38]:
combined_data.iloc[950:1000]

,ds,yhat,yhat_lower,yhat_upper,Close,model
950,2024-10-24,191.227964,16.835784,353.193260,NaN,prophet
951,2024-10-25,191.658581,14.046067,354.344310,NaN,prophet
952,2024-10-28,192.155807,15.020663,358.640611,NaN,prophet
953,2024-10-29,192.628843,12.947966,358.763525,NaN,prophet
954,2024-10-30,192.806490,11.951488,360.666919,NaN,prophet
955,2024-10-31,192.882821,10.121507,364.295672,NaN,prophet
956,2024-11-01,193.425637,8.657369,365.627943,NaN,prophet
957,2024-11-04,194.390809,6.688883,368.473877,NaN,prophet
958,2024-11-05,195.057951,6.333000,372.412429,NaN,prophet
959,2024-11-06,195.443312,5.648358,375.054952,NaN,prophet


In [40]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'stockdata' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'combined_data' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
combined_data.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


In [41]:
combined_data.to_csv('combinded_data3.csv', index=False)

# 모의계산기용

In [42]:
import pandas as pd
import yfinance as yf
start = '2021-01-01'
end = '2024-12-31'
symbol = 'AAPL' # "005930.KS"
df = yf.download(symbol, start, end)
df = df.rename(columns={'Adj Close': 'Adj_Close'})
# df['ds'] = pd.to_datetime(df.index, format = '%Y-%m-%d') # 굳이 넣어야하나 고민중
# df['y'] = df['Close'] # 마찬가지로 굳이 넣어야하나 고민중
df.head(3)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2021-01-04,133.520004,133.610001,126.760002,129.410004,127.002090,143301900
2021-01-05,128.889999,131.740005,128.429993,131.009995,128.572327,97664900
2021-01-06,127.720001,131.050003,126.379997,126.599998,124.244392,155088000


In [43]:
df['datestamp'] = pd.to_datetime(df.index, format = '%Y-%m-%d')
df_real = df[['datestamp', 'Close']]
df_real = df_real.rename(columns={'Close': 'y'})

In [44]:
df_real.reset_index(inplace=True, drop=True)

In [45]:
df_real[-1:]

,datestamp,y
799,2024-03-08,170.729996


In [3]:
df_predict = pd.read_csv('prophet_일년짜리예측3.csv')

In [47]:
df_predict=df_predict[df_predict['ds']>'2024-03-08'][['ds','yhat']]

In [48]:
df_predict.reset_index(inplace=True, drop=True)

In [49]:
df_predict.rename(columns={'ds': 'datestamp', 'yhat': 'y'}, inplace=True)

In [50]:
df_predict

,datestamp,y
0,2024-03-11,181.690492
1,2024-03-12,182.337799
2,2024-03-13,182.750906
3,2024-03-14,183.101862
4,2024-03-15,183.917855
...,...,...
199,2024-12-24,210.580028
200,2024-12-26,208.749040
201,2024-12-27,209.741494
202,2024-12-30,208.554550


In [51]:
df_real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   datestamp  800 non-null    datetime64[ns]
 1   y          800 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 12.6 KB


In [52]:
df_predict['datestamp']= pd.to_datetime(df_predict['datestamp'])

In [53]:
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   datestamp  204 non-null    datetime64[ns]
 1   y          204 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.3 KB


In [54]:
calculator_data=pd.concat([df_real, df_predict])

In [55]:
calculator_data

,datestamp,y
0,2021-01-04,129.410004
1,2021-01-05,131.009995
2,2021-01-06,126.599998
3,2021-01-07,130.919998
4,2021-01-08,132.050003
...,...,...
199,2024-12-24,210.580028
200,2024-12-26,208.749040
201,2024-12-27,209.741494
202,2024-12-30,208.554550


In [56]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'stockdata' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'calculator_data' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
calculator_data.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


# 예측 평가 지표 기록하기

In [58]:
start = '2021-01-01' # '2022-01-01'
end = '2023-12-31'
symbol = 'AAPL' # "005930.KS"
total_train_df = yf.download(symbol, start, end)
total_train_df = total_train_df.rename(columns={'Adj Close': 'Adj_Close'})
total_train_df['ds'] = pd.to_datetime(total_train_df.index, format = '%Y-%m-%d')
total_train_df['y'] = total_train_df['Close']

total_train_df_input = total_train_df[['ds','y']]

fcast_time = 30 # 365일 예측
freq_option = 'D'


# 문자열을 datetime 객체로 변환
end_date = datetime.strptime(end, '%Y-%m-%d')
start_date_r = end_date + timedelta(days=1) # fcast_time+1 만큼의 일수를 더함
end_date_r = end_date + timedelta(days=fcast_time+2) # fcast_time+1 만큼의 일수를 더함 end날 이전까지 가져와서 +1 해줌
start_r = start_date_r.strftime('%Y-%m-%d') # 문자로 바꿔줌
end_r = end_date_r.strftime('%Y-%m-%d') # 문자로 바꿔줌
ticker_r = 'AAPL'

df_recent = yf.download(ticker_r, start=start_r, end=end_r)
df_recent['ds'] = pd.to_datetime(df_recent.index, format = '%Y-%m-%d')
df_recent['y'] = df_recent['Close']

us_holidays = make_holidays_df(
    year_list=[2021 + i for i in range(4)], country='US'
)
us_holidays = us_holidays.assign(lower_window=-1, upper_window=1)
total_train_df_train_df_input = total_train_df[['ds','y']]
model_prophet = Prophet(holidays=us_holidays, changepoint_prior_scale=0.3,
                        holidays_prior_scale=10.0, n_changepoints=50, seasonality_mode='multiplicative', seasonality_prior_scale=0.01).fit(total_train_df_input)

future = model_prophet.make_future_dataframe(periods = fcast_time+3, freq = freq_option)
future = future[(future['ds'].dt.dayofweek < 5)&(~future['ds'].isin(us_holidays['ds']))] # 주말 제외 작업 & # 공휴일 제외 작업

df_forecast = model_prophet.predict(future)
# model_prophet.plot(df_forecast, xlabel = 'Date',ylabel ='price($)')
# model_prophet.plot_components(df_forecast);

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
18:44:27 - cmdstanpy - INFO - Chain [1] start processing
18:44:28 - cmdstanpy - INFO - Chain [1] done processing


In [59]:
df_forecast.set_index('ds', inplace=True, drop=False)
fc = df_forecast[df_forecast['ds'].isin(df_recent['ds'])]['yhat']

mse = mean_squared_error(df_recent['y'], fc)
mae = mean_absolute_error(df_recent['y'], fc)
rmse = math.sqrt(mean_squared_error(df_recent['y'], fc))
mape = mean_absolute_percentage_error(df_recent['y'], fc)
r2score = r2_score(df_recent['y'],fc)
print(f'mse: {mse}, mae: {mae}, rmse: {rmse}, mape: {mape}, r2_score: {r2score}')

mse: 28.172511393835574, mae: 4.576488756079693, rmse: 5.307778385900788, mape: 0.024566317094153974, r2_score: -0.4694172244245496


# 예측 평가 지표

In [60]:
metrics = pd.read_csv('metrics_prophet(final)3.csv')

In [61]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'predict_metrics' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'metrics_prophet_aapl' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
metrics.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]


In [62]:
# 아리마 예측 적재용
import pandas as pd
metrics_arima = pd.read_csv('metrics_arima.csv')

In [63]:
# metrics_prophet = metrics
metrics_prophet =pd.read_csv('metrics_prophet(final)3.csv')

In [64]:
metrics_prophet = metrics_prophet.assign(model='prophet')
metrics_arima = metrics_arima.assign(model='arima')

In [65]:
metrics_combined = pd.concat([metrics_prophet,metrics_arima])

In [66]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQuery 클라이언트 생성
client = bigquery.Client()
project_id = client.project #'actual-project-415811'

### 여기 설정 ###
dataset_name = 'predict_metrics' # 여기 부분 원하는 데이터셋으로 설정
### 여기 설정 끝 ###

# 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
dataset_id = "{}.{}".format(project_id, dataset_name)

# 데이터 세트 참조 생성
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

try:
    # 데이터 세트가 존재하는지 확인
    client.get_dataset(dataset_ref)
    print("Dataset already exists.")
except NotFound:
    # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
    dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
    dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
    dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
    print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

# client = bigquery.Client()
# dataset = 'stockdata' 

### 여기 설정 ###
table_name = 'metrics_combined_aapl' # 'predict_aapl_2021-01-01to2023-12-31_to365days'
### 여기 설정 끝 ###

table_id = "{}.{}".format(dataset_name,table_name)
metrics_combined.to_gbq(destination_table=table_id, project_id=project_id, if_exists='replace')

Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]
